# Cricket Data Exploratory Data Analysis

This notebook contains various visualizations analyzing cricket match data across different formats (Test, ODI, T20I, and IPL).

In [22]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from create_tables import DatabaseHandler

In [ ]:
def load_data():

    db = DatabaseHandler()
    
    queries = {
        'test_matches': 'SELECT * FROM test_matches',
        'odi_matches': 'SELECT * FROM odi_matches',
        't20_matches': 'SELECT * FROM t20_matches',
        'ipl_matches': 'SELECT * FROM ipl_matches',
        'test_deliveries': 'SELECT * FROM test_deliveries',
        'odi_deliveries': 'SELECT * FROM odi_deliveries',
        't20_deliveries': 'SELECT * FROM t20_deliveries',
        'ipl_deliveries': 'SELECT * FROM ipl_deliveries'
    }
    
    dataframes = {}
    for table_name, query in queries.items():
        try:
            dataframes[table_name] = pd.read_sql(query, db.connection)
        except Exception as e:
            print(f"Error loading {table_name}: {e}")
    
    db.close_connection()
    return dataframes

print("Loading data from database...")
data = load_data()

### Distribution of Matches Across Different Formats

In [ ]:
format_counts = {
    'Test': len(data['test_matches']),
    'ODI': len(data['odi_matches']),
    'T20I': len(data['t20_matches']),
    'IPL': len(data['ipl_matches'])
}

plt.figure(figsize=(10, 6))
colors = ['#2ecc71', '#3498db', '#e74c3c', '#f1c40f']
plt.bar(format_counts.keys(), format_counts.values(), color=colors)
plt.title('Distribution of Matches Across Different Formats', pad=20)
plt.ylabel('Number of Matches')
for i, v in enumerate(format_counts.values()):
    plt.text(i, v + 30, str(v), ha='center')
plt.show()

### Toss Decisions Across Formats

In [ ]:
formats = ['test_matches', 'odi_matches', 't20_matches', 'ipl_matches']
toss_data = []

for format_name in formats:
    df = data[format_name]
    toss_counts = df['toss_decision'].value_counts()
    for decision, count in toss_counts.items():
        toss_data.append({
            'Format': format_name.split('_')[0].upper(),
            'Decision': decision.capitalize(),
            'Count': count
        })

df_toss = pd.DataFrame(toss_data)
fig = px.bar(df_toss, x='Format', y='Count', color='Decision',
             title='Toss Decisions Across Formats',
             barmode='group')
fig.show()

### Win Percentage After Winning Toss

In [ ]:
formats = ['test_matches', 'odi_matches', 't20_matches', 'ipl_matches']
win_percentages = []

for format_name in formats:
    df = data[format_name]
    winner_col = 'winner'
    if format_name == 'test_matches':
        winner_col = 'outcome_winner'
    
    toss_winners = df[df['toss_winner'] == df[winner_col]].shape[0]
    total_matches = df.shape[0]
    win_percentages.append((toss_winners / total_matches) * 100)

plt.figure(figsize=(12, 6))
sns.barplot(x=[f.split('_')[0].upper() for f in formats], y=win_percentages)
plt.title('Win Percentage After Winning Toss')
plt.ylabel('Win Percentage')
for i, v in enumerate(win_percentages):
    plt.text(i, v + 1, f'{v:.1f}%', ha='center')
plt.show()

### Runs Distribution Across Formats

In [ ]:
formats = ['test_deliveries', 'odi_deliveries', 't20_deliveries', 'ipl_deliveries']
runs_list = []
format_list = []

for format_name in formats:
    df = data[format_name]
    runs_list.extend(df['runs_total'].tolist())
    format_list.extend([format_name.split('_')[0].upper()] * len(df))

df_violin = pd.DataFrame({
    'Format': format_list,
    'Runs': runs_list
})

plt.figure(figsize=(12, 6))
sns.violinplot(x='Format', y='Runs', data=df_violin)
plt.title('Distribution of Runs Scored per Ball Across Formats')
plt.ylabel('Runs per Ball')
plt.show()

### Distribution of Winning Margins in IPL

In [ ]:
ipl_df = data['ipl_matches'].copy()

ipl_df['win_margin_type'] = ipl_df.apply(lambda row: 'Runs' if row.get('win_by_runs', 0) > 0 
                                           else ('Wickets' if row.get('win_by_wickets', 0) > 0 else np.nan), axis=1)

runs_margin = ipl_df[ipl_df['win_margin_type'] == 'Runs']['win_by_runs']
wickets_margin = ipl_df[ipl_df['win_margin_type'] == 'Wickets']['win_by_wickets']

plt.figure(figsize=(12, 5))

plt.subplot(1,2,1)
sns.histplot(runs_margin, bins=20, kde=True, color='skyblue')
plt.title('Distribution of Win Margins by Runs')
plt.xlabel('Win Margin (Runs)')

plt.subplot(1,2,2)
sns.histplot(wickets_margin, bins=20, kde=True, color='salmon')
plt.title('Distribution of Win Margins by Wickets')
plt.xlabel('Win Margin (Wickets)')

plt.tight_layout()
plt.show()

### Distribution of Wicket Types

In [ ]:
wicket_data = []
for format_name in ['test_deliveries', 'odi_deliveries', 't20_deliveries', 'ipl_deliveries']:
    df = data[format_name]
    wickets = df[df['wicket_kind'].notna()]['wicket_kind'].value_counts()
    for wicket_type, count in wickets.items():
        wicket_data.append({
            'Format': format_name.split('_')[0].upper(),
            'Wicket Type': wicket_type,
            'Count': count
        })

df_wickets = pd.DataFrame(wicket_data)

fig = px.bar(
    df_wickets,
    x="Format",
    y="Count",
    color="Wicket Type",
    title="Distribution of Wicket Types Across Formats",
    barmode="group"
)
fig.show()

### Comparison of Extras Across Formats

In [ ]:
extras_data = []
for format_name in ['test_deliveries', 'odi_deliveries', 't20_deliveries', 'ipl_deliveries']:
    df = data[format_name]
    extras = {
        'Format': format_name.split('_')[0].upper(),
        'Wides': df['extras_wides'].sum(),
        'No Balls': df['extras_noballs'].sum(),
        'Byes': df['extras_byes'].sum(),
        'Leg Byes': df['extras_legbyes'].sum()
    }
    extras_data.append(extras)

df_extras = pd.DataFrame(extras_data)
fig = go.Figure()
for extra_type in ['Wides', 'No Balls', 'Byes', 'Leg Byes']:
    fig.add_trace(go.Bar(name=extra_type, x=df_extras['Format'], y=df_extras[extra_type]))

fig.update_layout(barmode='stack', title='Comparison of Extras Across Formats')
fig.show()

### Seasonal Trend of Matches

In [ ]:
plt.figure(figsize=(15, 6))
for format_name in ['test_matches', 'odi_matches', 't20_matches', 'ipl_matches']:
    df = data[format_name]
    df['year'] = pd.to_datetime(df['date']).dt.year
    yearly_counts = df['year'].value_counts().sort_index()
    plt.plot(yearly_counts.index, yearly_counts.values, label=format_name.split('_')[0].upper(), marker='o')

plt.title('Seasonal Trend of Matches')
plt.xlabel('Year')
plt.ylabel('Number of Matches')
plt.legend()
plt.grid(True)
plt.show()

### Top Teams in International Cricket

In [ ]:
team_wins = {}
for format_name in ['test_matches', 'odi_matches', 't20_matches']:
    df = data[format_name]

    if format_name == 'test_matches':
        wins = df['outcome_winner'].value_counts()
    else:
        wins = df['winner'].value_counts()
        
    for team, count in wins.items():
        team_wins[team] = team_wins.get(team, 0) + count

top_teams = dict(sorted(team_wins.items(), key=lambda x: x[1], reverse=True)[:10])

plt.figure(figsize=(12, 6))
colors = sns.color_palette('husl', n_colors=len(top_teams))
plt.pie(top_teams.values(), labels=top_teams.keys(), colors=colors, autopct='%1.1f%%')
plt.title('Top 10 Teams by Wins in International Cricket')
plt.show()

### IPL Winning Methods by Season

In [ ]:
ipl_df = data['ipl_matches'].copy()
ipl_df['season'] = pd.to_datetime(ipl_df['date']).dt.year

def decide_margin(row):
    if row.get('win_by_runs', 0) > 0:
        return 'Runs'
    elif row.get('win_by_wickets', 0) > 0:
        return 'Wickets'
    else:
        return 'Other'

ipl_df['decided_by'] = ipl_df.apply(decide_margin, axis=1)

margin_trend = ipl_df.groupby(['season', 'decided_by']).size().reset_index(name='count')

margin_trend_pivot = margin_trend.pivot(index='season', columns='decided_by', values='count').fillna(0)

plt.figure(figsize=(12, 6))
margin_trend_pivot.plot(kind='bar', stacked=True, colormap='viridis', ax=plt.gca())
plt.title('Trend of IPL Match Winning Methods by Season')
plt.xlabel('Season')
plt.ylabel('Number of Matches')
plt.legend(title='Decided By')
plt.tight_layout()
plt.show()